# ML Test 2

In [40]:
import tensorflow as tf
import numpy as np 
from PIL import Image

In [2]:
# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode='fine')

In [3]:
# Normalize the pixel values
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert the labels to binary arrays using one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=100)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=100)

In [16]:
# Define the categories we want to use
house_category = 55  # the "house" category
animal_categories = [26, 27, 28, 29, 30, 31, 32, 33]  # a subset of the "large omnivores and herbivores" category

# Select only the images and labels corresponding to the desired categories
x_train_house = x_train[y_train[:, house_category] == 1]
y_train_house = np.zeros((500, 2))
y_train_house[:, 0] = 1 # Assign the "House" category
np.random.seed(42)
idx = np.random.choice(4000, size=500, replace=True)
x_train_animals = x_train[y_train[:, animal_categories].sum(axis=1) == 1][idx]
y_train_animals = np.zeros((500, 2))
y_train_animals[:, 1] = 1 # Assign the "Animals" category

# Combine the selected data into a single training set
x_train_combined = tf.concat([x_train_house, x_train_animals], axis=0)
y_train_combined = tf.concat([y_train_house, y_train_animals], axis=0)

In [29]:
# Select testing data the same way
x_test_house = x_test[y_test[:, house_category] == 1]
y_test_house = np.zeros((100, 2))
y_test_house[:, 0] = 1
idx = np.random.choice(800, size=1, replace=False)
x_test_animals = x_test[y_test[:, animal_categories].sum(axis=1) == 1][idx]
y_test_animals = np.zeros((100, 2))
y_test_animals[:, 1] = 1

x_test_combined = tf.concat([x_test_house, x_test_animals], axis=0)
y_test_combined = tf.concat([y_test_house, y_test_house], axis=0)

In [22]:
# Create a sequential model
model = tf.keras.models.Sequential()

# Add a 2D convolutional layer with 32 filters, a 3x3 kernel, and 'same' padding
model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=x_train.shape[1:]))

# Add a max pooling layer with a 2x2 pool size
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# Add another 2D convolutional layer with 64 filters and a 3x3 kernel
model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'))

# Flatten the output from the convolutional layers
model.add(tf.keras.layers.Flatten())

# Add a dense layer with 128 units and 'relu' activation
model.add(tf.keras.layers.Dense(128, activation='relu'))

# Add an output layer with 2 units and 'softmax' activation
model.add(tf.keras.layers.Dense(2, activation='softmax'))

In [23]:
# Compile the model with 'adam' optimizer, 'categorical_crossentropy' loss, and 'accuracy' metric
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
tf.config.optimizer.set_jit(False)

In [33]:
# Train the model on the training data
model.fit(x_train_combined, y_train_combined, batch_size=32, epochs=10, validation_data=(x_test_combined, y_test_combined))

Epoch 1/10


TypeError: 'NoneType' object is not callable